<a href="https://colab.research.google.com/github/albhoe/701_assignments/blob/main/911Call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')